In [40]:
import pandas as pd
import re
from openpyxl import load_workbook

In [42]:
import re
import pandas as pd

path = "D:/engenharia de dados/files/whats/networking.txt"

with open(path, "r", encoding="utf-8") as f:
    content = f.readlines()

# Captura só a HORA; a DATA vira não-capturante (?: ...)
default = r"^(?:\d{1,2}\/\d{1,2}\/\d{4}) (\d{2}:\d{2}) - (?:(.*?): )?(.*)$"
# Variante mais tolerante (aceita ano com 2 ou 4 dígitos e vírgula após a data):
# default = r"^(?:\d{1,2}\/\d{1,2}\/\d{2,4}),?\s+(\d{2}:\d{2}) - (?:(.*?): )?(.*)$"

data = []
current_message = None

for line in content:
    match = re.match(default, line)
    if match:
        hour, sender, message = match.groups()
        current_message = [hour, sender if sender else "SISTEMA", message]
        data.append(current_message)
    else:
        if current_message:
            current_message[2] += "\n" + line 

df = pd.DataFrame(data, columns=["Hora", "Remetente", "Mensagem"])

print(df.head(50))


     Hora          Remetente  \
0   10:09            SISTEMA   
1   19:08            SISTEMA   
2   19:08            SISTEMA   
3   19:08            SISTEMA   
4   10:09            SISTEMA   
5   10:15            SISTEMA   
6   10:22            SISTEMA   
7   10:22            SISTEMA   
8   10:24            SISTEMA   
9   10:26            SISTEMA   
10  10:26            SISTEMA   
11  10:26            SISTEMA   
12  10:28            SISTEMA   
13  10:28            SISTEMA   
14  10:29            SISTEMA   
15  10:32            SISTEMA   
16  10:34            SISTEMA   
17  10:36            SISTEMA   
18  10:43  +55 21 96989-7976   
19  10:44            SISTEMA   
20  10:45            SISTEMA   
21  10:46            SISTEMA   
22  10:49            SISTEMA   
23  10:56            SISTEMA   
24  10:58            SISTEMA   
25  11:03            SISTEMA   
26  11:07            SISTEMA   
27  11:12            SISTEMA   
28  11:12            SISTEMA   
29  11:12            SISTEMA   
30  11:1

In [43]:
df_linkedin = df[df["Mensagem"].str.contains(r"linkedin\.com", case=False, na=False)].copy()

df_linkedin["Mensagem"] = df_linkedin["Mensagem"].str.extract(
    r"((?:https?://)?(?:www\.)?linkedin\.com[^\s]+)",
    expand=False
)

df_linkedin["Mensagem"] = df_linkedin["Mensagem"].apply(
    lambda x: "https://" + x if pd.notnull(x) and not x.startswith("http") else x
)

df_linkedin.rename(columns={"Mensagem": "Linkedin"}, inplace=True)

print(df_linkedin[["Hora", "Remetente", "Linkedin"]])


      Hora          Remetente  \
168  12:00   +55 71 9308-2437   
169  12:00   +55 93 9138-9543   
170  12:00   +55 31 9219-4008   
176  12:01   +55 81 9998-2162   
178  12:01  +55 11 97020-9285   
..     ...                ...   
611  14:04   +55 85 8808-3993   
613  14:04   +55 48 9925-8353   
614  14:05   +55 82 9933-5023   
616  14:06   +55 88 9704-3295   
617  14:06  +55 27 99938-2967   

                                                                                                                                              Linkedin  
168                                                                                                        https://www.linkedin.com/in/iasmim-horrana/  
169                                                                                                            https://www.linkedin.com/in/max-mitsuya  
170           https://www.linkedin.com/in/jo%C3%A3o-victor-1718b567?utm_source=share&utm_campaign=share_via&utm_content=profile&utm_medium=andro

In [46]:
path_excel = r"D:\\engenharia de dados\\files\\whats\\networking.xlsx"

df_linkedin.to_excel(path_excel, index=False)

wb = load_workbook(path_excel)
ws = wb.active

for row in range(2, ws.max_row + 1):
    cell = ws[f"C{row}"]
    url = cell.value
    if url:
        cell.hyperlink = url
        cell.value = "Linkedin"
        cell.style = "Hyperlink"

wb.save(path_excel)
